In [29]:
from collections import Counter
from imblearn.over_sampling import SMOTE
import matplotlib                  # 2D Plotting Library
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
from sklearn.pipeline import make_pipeline
from lime import lime_text
from lime.lime_text import LimeTextExplainer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer  
from utils import preprocess_text

#Paste the test data to test on the model
file ="train.csv"
df = pd.read_csv(file)

# DATA HANDLING BEFORE XAI ANALYSIS

In [ ]:
#############
df['processed_text'] = df['text'].apply(preprocess_text)

In [ ]:
#Loading x_test and y_test
X_test = df['processed_text']
y_test = df['label']

vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_df=0.95, min_df=5)
vect = CountVectorizer(min_df=5)  # Convert a collection of text documents to a matrix of token counts.
X_test_tok = vect.transform(X_test)

# LOAD THE SVM MODEL

In [ ]:
#############
clf = ''
predi = clf.predict(X_test)

In [ ]:
print(f"Accuracy: {accuracy_score(y_test, predi)}")

print("Classification Report:")
print(classification_report(y_test, predi))

In [ ]:
c = make_pipeline(vectorizer, clf)

# Making explainable predictions

In [ ]:
print('Text:'+df["text"][0])
class_names = c.classes_
probabilities = c.predict_proba([df["text"][0]])[0]
label_probabilities = dict(zip(class_names, probabilities))
print('predict_proba:'+ str(label_probabilities))

In [ ]:
explainer = LimeTextExplainer(class_names=class_names)

X_test = X_test.tolist()
y_test = y_test.tolist()

### CHECK WHAT LABEL YOU WANT FOR THE XAI EXPLANAITION

In [ ]:
#printing ids of comments which are toxic
""" count=-1
for x in y_test:
    count=count+1
    if x==1:
        print(count) """

# FOR LABEL N (of all the emotions)

In [ ]:
idx = 26
exp_nontoxic = explainer.explain_instance(X_test[idx], c.predict_proba, num_features=10)
print('Document id: %d' % idx)
print('Probability(toxic) =', c.predict_proba([X_test[idx]])[0,1])
print('True class: %s' % class_names[y_test[idx]])

In [ ]:
exp_nontoxic.as_list()

In [ ]:
print('Original prediction:', clf.predict_proba(X_test_tok[idx])[0,1])
tmp = X_test_tok[idx].copy()
########CHANGE THE WORDS THAT YOU WANT TO ANALIZE FROM TEXT
tmp[0,vectorizer.vocabulary_['talk']] = 0
tmp[0,vectorizer.vocabulary_['pretending']] = 0
print('Prediction removing some features:', clf.predict_proba(tmp)[0,1])
print('Difference:', clf.predict_proba(tmp)[0,1] - clf.predict_proba(X_test_tok[idx])[0,1])

In [ ]:
%matplotlib inline
fig = exp_nontoxic.as_pyplot_figure()

In [ ]:
exp_nontoxic.show_in_notebook()